# Exercise: NEISS, Question Set C

#### Summary

The [National Electronic Injury Surveillance System](https://www.cpsc.gov/Safety-Education/Safety-Guides/General-Information/National-Electronic-Injury-Surveillance-System-NEISS) is a data product produced by the US Consumer Product Safety Commission. It tracks emergency room injuries related to consumer products (e.g., "a door fell on me!").

#### Files

- **neiss2017.tsv**: injury data (one injury per row)
- **2018-NEISS-CPSC-only-CodingManual.pdf**: column definitions and explanations
- **2017 NEISS Data Highlights.pdf**: a partial summary of the data
- **2018ComparabilityTable.pdf**: product code definitions
- **categories-cleaned.txt**: product code definitions in CSV format (great for joining!)

#### Source

https://www.cpsc.gov/Safety-Education/Safety-Guides/General-Information/National-Electronic-Injury-Surveillance-System-NEISS

#### Skills

- Reading in files
    - Reading tab-separated files
    - Reading in N/A values
    - Only reading in some of the data
- Replacing values
- Using strings
    - Searching for strings
    - Comparing to a list of strings
    - Regular expressions
- Using numpy/`np.nan`
- Averages practice
- Converting `.value_counts()` and similar results into DataFrames

# Read in `neiss2017.tsv`

Something's... weird about this one.

In [1]:
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
#Use a tab as what's separating the values 
df = pd.read_csv('neiss2017.tsv', sep = '\t', encoding='latin1')
df.head()
#windows1 is also popular 

C:\Users\kellykiki\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight
0,170100733,01/01/2017,21,1,1.0,NaN,75.0,62.0,NaN,1.0,1.0,0.0,1207.0,0.0,21YOM WAS BOXING WITH FRIEND WHILE DRINKING AN...,. BAC= 93.6 DX HEAD INJURY AND ALCOHOL INTOXIC...,S,46.0,77.6641
1,170100734,01/01/2017,71,1,1.0,NaN,31.0,53.0,NaN,1.0,5.0,0.0,1807.0,0.0,71YOM FELL ON FLOOR IN BATHROOM. DX RIB CONTUSION,NaN,S,46.0,77.6641
2,170100735,01/01/2017,16,1,1.0,NaN,33.0,51.0,NaN,1.0,1.0,0.0,676.0,342.0,16YOM TRIPPED OVER A RUG AND FELL INTO A BURNI...,DX BURNS INVOLVING LESS THAN 10% OF BODY SURFA...,S,46.0,77.6641
3,170100737,01/01/2017,55,2,1.0,NaN,75.0,62.0,NaN,1.0,1.0,0.0,620.0,0.0,55YOF WAS STANDING ON A STEP STOOL AND LOST FO...,HEAD INJURY,S,46.0,77.6641
4,170100740,01/01/2017,86,2,1.0,NaN,79.0,58.0,NaN,1.0,1.0,0.0,1646.0,4056.0,86YOF GOT TANGLED IN COAT AND FELL BACKWARD ST...,"H. DX HEMATOMA OF SOFT TISSUE, LEFT FLANK",S,46.0,77.6641


### Check that your dataframe has 386907 rows and 19 columns.

In [4]:
df.shape

(386907, 19)

### List the columns and their data types

In [5]:
df.dtypes

CPSC_Case_Number     object
Treatment_Date       object
Age                  object
Sex                   int64
Race                float64
Other_Race           object
Body_Part           float64
Diagnosis           float64
Other_Diagnosis      object
Disposition         float64
Location            float64
Fire_Involvement    float64
Product_1           float64
Product_2           float64
Narrative_1          object
Narrative_2          object
Stratum              object
PSU                 float64
Weight              float64
dtype: object

### I've selected a few columns. What do they mean?

The columns you are interested in are...

- `CPSC_Case_Number`
- `Race`
- `Other_Race`
- `Product_1`
- `Product_2`

You'll need to use the **coding manual**, `2018-NEISS-CPSC-only-CodingManual.pdf`. Column definitions are all in the first 30 pages or so. I recommend using the table of contents.

# Cleaning up a column

Take a look at the **Race** column. How many rows of each race are there?

In [6]:
df.Race

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
5         1.0
6         1.0
7         1.0
8         1.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
386877    0.0
386878    1.0
386879    3.0
386880    1.0
386881    1.0
386882    1.0
386883    0.0
386884    0.0
386885    3.0
386886    1.0
386887    1.0
386888    1.0
386889    1.0
386890    2.0
386891    1.0
386892    1.0
386893    1.0
386894    1.0
386895    1.0
386896    1.0
386897    1.0
386898    1.0
386899    1.0
386900    1.0
386901    2.0
386902    1.0
386903    2.0
386904    1.0
386905    1.0
386906    1.0
Name: Race, Length: 386907, dtype: float64

In [7]:
df.Race.value_counts()

1.000    160527
0.000    143677
2.000     55144
3.000     21193
4.000      4714
5.000      1234
6.000       417
4.757         1
Name: Race, dtype: int64

## Replace the numbers with the appropriate words they stand for.

Those numbers are terrible - codes are fine for storage but not really for reading. We want to **replace the numbers with the words they stand for.**

In [8]:
#This creats a new column which lives in an outer space. We can save it a new column but...
df.Race.replace(1, 'White')

0         White
1         White
2         White
3         White
4         White
5         White
6         White
7         White
8         White
9             0
10            0
11            0
12            0
13            0
14            0
15            0
16            0
17            0
18            0
19            0
20            0
21            0
22            0
23            0
24            0
25            0
26            0
27            0
28            0
29            0
          ...  
386877        0
386878    White
386879        3
386880    White
386881    White
386882    White
386883        0
386884        0
386885        3
386886    White
386887    White
386888    White
386889    White
386890        2
386891    White
386892    White
386893    White
386894    White
386895    White
386896    White
386897    White
386898    White
386899    White
386900    White
386901        2
386902    White
386903        2
386904    White
386905    White
386906    White
Name: Race, Length: 3869

In [9]:
df.Race.replace(1, 'White', inplace = True)

In [10]:
df.Race.value_counts().head()

White    160527
0.0      143677
2.0       55144
3.0       21193
4.0        4714
Name: Race, dtype: int64

In [11]:
df.Race.replace(2, 'Black/African American', inplace = True)

In [12]:
df.Race.value_counts().head()

White                     160527
0.0                       143677
Black/African American     55144
3.0                        21193
4.0                         4714
Name: Race, dtype: int64

In [31]:
#key is original code
#value is replacement
df.Race.replace({
    1: 'White',
    2: 'Black/African American',
    4: 'Asian',
    5: 'American Indian/Alaska Native',
    6: 'Native Hawaiian/Pacific Islander',
    3: 'Other',
    0: 'Not Stated in ED record'
}, inplace = True)

In [32]:
df.Race

0                           White
1                           White
2                           White
3                           White
4                           White
5                           White
6                           White
7                           White
8                           White
9         Not Stated in ED record
10        Not Stated in ED record
11        Not Stated in ED record
12        Not Stated in ED record
13        Not Stated in ED record
14        Not Stated in ED record
15        Not Stated in ED record
16        Not Stated in ED record
17        Not Stated in ED record
18        Not Stated in ED record
19        Not Stated in ED record
20        Not Stated in ED record
21        Not Stated in ED record
22        Not Stated in ED record
23        Not Stated in ED record
24        Not Stated in ED record
25        Not Stated in ED record
26        Not Stated in ED record
27        Not Stated in ED record
28        Not Stated in ED record
29        Not 

## Confirm you have 160527 White, 143677 not stated, and 55144 Black.

There's one person with a real strange race, but we'll save them for later.

In [33]:
df[df.Race == 160527]

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


In [34]:
df[df.Race == 143677]

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


In [35]:
df[df.Race == 55144]

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


In [36]:
df[df.Race == 'Nan']

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


In [37]:
# .isnull() is what you feel like should be == 'Nan'
df[df.Race.isnull()]

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


In [38]:
#Drop all rows where 'Race' is missing data
df.dropna(subset=['Race'], inplace=True)

In [40]:
#df[df.Race

## Graph the number of each race, but don’t included the “Not Stated” records

## "Not Stated" seems silly - change it to be `NaN` instead

Don't use `na_values` for this.

In [41]:
df.Race.value_counts()

White                               160527
Not Stated in ED record             143677
Black/African American               55144
Other                                21193
Asian                                 4714
American Indian/Alaska Native         1234
Native Hawaiian/Pacific Islander       417
4.757                                    1
Name: Race, dtype: int64

In [42]:
import numpy as np
df.Race.replace(0, np.nan, inplace=True)

In [43]:
np.nan

nan

In [44]:
df.Race.value_counts()

White                               160527
Not Stated in ED record             143677
Black/African American               55144
Other                                21193
Asian                                 4714
American Indian/Alaska Native         1234
Native Hawaiian/Pacific Islander       417
4.757                                    1
Name: Race, dtype: int64

### What about that weird value? Look at the row where the race is `4.757`

In [45]:
df[df.Race == '4.757']

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


## Let's drop that bad row

It looks like bad data! Maybe we can drop it based on `Treatment_Date` being null? 

### How many times is `Treatment Date` been empty?

### Drop the row where `Treatment_Date` is empty

### Confirm you have no more missing treatment dates

## Graph the count of each race, but don’t included the “Not Stated” records

Yes, again! The code you use should be different this time.

# Cleaning up `Other_Race`

## `Other_Race` is a field for free-form race input. How many patients have a race of "HISPANIC"?

## What are the most popular "other" races? The least popular?

Oh, just do a `.value_counts()`

## Searching for multiracial patients

Wow, this data entry is terrible. “Multiracial” is spelled as **MULT RACIAL**, **MULTIPLE RACIAL**, and many more. How many different spellings can you find? **Use only one line to find the spellings.**

- Tip: Be sure to **ignore the na values**.
- Tip: You should probably find the multiracial-ish rows and then `value_counts` their `other_race`
- Tip: Maybe... ask me about .str.contains support for regular expressions?

In [ ]:
#_____WALL_____PUNCH____ - 
#_____MUL______RAC______ -

In [52]:
df.[df.Other_Race.str.]

AttributeError: 'DataFrame' object has no attribute 'Other'

## Replace all of those spellings with “MULTIRACIAL.”

Confirm that you’ve ended up with about 2440 MULTIRACIAL rows (yours might be anywhere between 2400-2500, depending on how many spellings you caught)

In [ ]:
df[df.Other_Race.str.contains(‘MUL.*RAC’, na = False)].replace(‘MUL.*RAC’,‘MULTIRACIAL’, inplace=True)

In [53]:
df.Other_Race.str.replace('MUL.*RAC','MULTIRACIAL',inplace=True)
df.Other_Race.value_counts()

TypeError: replace() got an unexpected keyword argument 'inplace'

In [54]:
#.'replace' vs. '.str.replace'
df['Other_Race'] = df.Other_Race.str.replace('.*MUL.*RAC.*', 'MULTIRACIAL', regex = True).inplace=True)
df.Other_Race.value_counts()

SyntaxError: invalid syntax (<ipython-input-54-0741254d01c4>, line 2)

## What variations on HISPANIC can you find? Standardize them to HISPANIC.

## Now try counting the number of Hispanic patients again.

## Graph the frequency of each "Other" race in the dataset

### That doesn't look nice. Maybe just the top 5?

## "NS" probably means "Not Stated" and "Unknown" probably doesn't mean anything - convert them to `NaN`

And keep replacing 'weird' or 'bad' values with `NaN` until you get a good top 10.

# Finding injuries

## How many people were injured by "Minibikes, powered?"

You'll use the codes at the end of the Coding Manual.

- Tip: `Product_1` and `Product_2` are the product fields

In [47]:
df.Race.str.contains("Product_1", 'Product_2')

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
386877    False
386878    False
386879    False
386880    False
386881    False
386882    False
386883    False
386884    False
386885    False
386886    False
386887    False
386888    False
386889    False
386890    False
386891    False
386892    False
386893    False
386894    False
386895    False
386896    False
386897    False
386898    False
386899    False
386900    False
386901    False
386902    False
386903    False
386904    False
386905    False
386906    False
Name: Race, Length: 3869

In [50]:
df[df.Race.str.contains("Product_1", 'Product_2', na=False)]

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight


In [ ]:
#______WALL_____PUNCH_______ - 
#______MUL______RAC_________ - 

In [ ]:
#Fill in the blanks 

## How many people were injured by "Musical instruments, electric or battery operated?"

Something will be... weird about this one. It might not even work for you!

## Fixing our reading-in process

Oh no, it looks like leading zeroes were stripped - instead of `"0566"` it's `556`. Why did pandas change it for us?

Can we go back and read in the file again and fix this problem?

## Hehehe oh no. Let's pad with zeroes manually using `zfill`

* Tip: Can you ask `zfill` for help?
* Tip: We need to **overwrite** the `Product_1` column

### Okay, now try counting again

## Get meaningful names for "product code"

Go clean `categories-exported.txt` and save it as `categories-cleaned.csv`. I made another notebook for you!

When you're done, we'll use this to turn the codes into actual words.

### Read in `categories-cleaned.csv` and make sure it looks okay

**It probably doesn't.** Go back to the other notebook and work on it until it looks right.

## Merge together the two datasets

This will use `merge`, but it's really just like an SQL join. Is that exciting? I don't know.

### Confirm that it worked by searching for every injury involving a skateboard

## What are the most popular product types for injuries involving musical instruments?

## Graph the top 30 most popular products for injuries involving people under 20

## Graph the top 30 most popular products for injuries involving people over 50 (but not over 100)